In [ ]:
import re, json, pathlib, logging, time, argparse
import datetime as dt
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from whoosh.lang.porter import stem
from itertools import chain

import fromRtoPython as SDGmapper

##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [ ]:
TEI_mapping = pathlib.Path.cwd() / 'output' / 'SA_and_Kenya_TEI_2021-02-18_T114717' / 'results' / 'mapping_SA_and_Kenya_TEI_2021-02-18_T114717.xlsx'

targets = pd.read_excel(TEI_mapping, index_col=0, sheet_name= 'Target_raw_count')
goals = pd.read_excel(TEI_mapping, index_col=0, sheet_name= 'Goal_raw_count')
targets['Goal'] = targets['Target'].apply(lambda x: str('SDG '+x.split('.')[0]))
targets.drop(labels='Target', axis=1, inplace=True)
targets = targets[['Policy', 'Goal', 'Keyword', 'Count', 'Textlength']]
total = pd.concat([targets, goals])
total = total.groupby(['Policy','Goal']).sum()

total.to_csv('TEI_by_goal.csv')
total.to_json('TEI_by_goal.json')

total.head(34)

In [ ]:
colors = ["#E5243B","#DDA63A","#4C9F38","#C5192D",
"#FF3A21","#26BDE2","#FCC30B","#A21942",
"#FD6925","#DD1367","#FD9D24","#BF8B2E",
"#3F7E44","#0A97D9","#56C02B","#00689D","#19486A"]

sdg_colors = {f'SDG {count}' : color for count, color in enumerate(colors)}

bubble_chart = BubbleChart(area=total['Goal'],
                           bubble_spacing=0.1)

bubble_chart.collapse()

fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))
bubble_chart.plot(
    ax, browser_market_share['Goal'], sdg_colors[browser_market_share['Goal']])
ax.axis("off")
ax.relim()
ax.autoscale_view()
ax.set_title('Browser market share')

plt.show()

In [ ]:
def save_text(dest_folder, filename, text_string):

    #dest folder must be an existing path
    
    filename_parent = filename.parent

    doctext_name = dest_folder.joinpath(filename_parent) 
    doctext_name.mkdir(mode=0o777, parents=True, exist_ok=True)
    doctext_name = doctext_name / str(filename.name.replace('.','_')+'.txt')

    with open(doctext_name, 'w', encoding='utf-8') as file_:
            file_.write(text_string)

In [ ]:
policy_documents['Doc_text']=policy_documents['Input_files'].apply(doc2text)
#policy_documents.apply(lambda col: save_text(doctext_dir, col['Paths'], col['Doc_text']))

for filename, text in zip(policy_documents['Paths'], policy_documents['Doc_text']):
    save_text(doctext_dir, filename, text)

# # PDFtext = []
# # counter = 0
# for doc_path in files:
#     counter += 1
#     try:
#         policy_text=[]
#         doc_text = doc2text(doc_path)
#         while '\n\n\n\n' in doc_text : doc_text = doc_text.replace('\n\n\n\n', '\n\n\n') #docx2python specific fix. would probably fit better elsewhere
#         policy_text.append(doc_text)
#         doctext_ = doc_path.parts[doc_path.parts.index(input_dir.name)+1:]
#         doctext_name =  doctext_dir.joinpath(*doctext_)
#         doctext_name.parent.mkdir(mode=0o777, parents=True, exist_ok=True)
#         doctext_name = doctext_name.parent.joinpath(doctext_name.name.replace('.','_')+'.txt')
#         with open(doctext_name, 'w', encoding='utf-8') as file_:
#            file_.write(doc_text)
#         PDFtext.append(['/'.join(doctext_),' ; '.join(policy_text)])
#     except Exception as excptn: #MM I'd log errors as described in https://realpython.com/python-logging/, we need to test this.
#         print(excptn)
#         logging.exception('{doc_file} raised exception: {exception} \n\n'.format(doc_file=doc_path.name, exception=excptn))


# with open(log_file, 'a') as f:
#     f.write( 
#         '3) Reading, converting and saving {docs} documents as text: {seconds:.3e} seconds\n\n'.format(docs=len(PDFtext),seconds=(time.time()-start_time))
#     )

# print('Step 3: Converted {docs} documents to text.\n'.format(docs=len(PDFtext)))